<a href="https://colab.research.google.com/github/fmarol/Assignments/blob/master/ST446_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


import numpy as np

## Assignment 1

We start by setting up the bucket and cluster. For part 1 & 2 we need to use the author-large.txt file, so we download this from the url link and place it in our bucket. 

The steps taken to do this are as follows:


The code ran to do this can be seen in Image 1 and 2.

### Part 1
First we 


In [0]:
data_from_file = sc.\
    textFile("gs://dora-bucket/author-large.txt", 4)

In [0]:
data_from_file.take(5)

[u'Jurgen Annevelink\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Rafiul Ahad\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Amelia Carlson\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Daniel H. Fishman\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Michael L. Heytens\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995']

In [0]:
import numpy as np
data_from_file_conv = data_from_file.map(lambda row: np.array(row.strip().split("\t")))
data_from_file_conv.take(5)

[array([u'Jurgen Annevelink', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Rafiul Ahad', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Amelia Carlson', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Daniel H. Fishman', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Michael L. Heytens', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78')]

In [0]:
#take only the Name and Title
book_authors = data_from_file_conv.map(lambda row: (row[2], row[0]))
#join the dataset on itself along the Title so that we have all possible combinations of pairs of authors for each book
book_authors_join = book_authors.join(book_authors).filter(lambda row: row[1][0]<row[1][1])
book_authors_join.take(5)

[(u'Impact of the Processing Methods on the Performance of the X-ray Film Screen Combinations.',
  (u'Cao Hou-de', u'Wang Yong-ming')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Peng Yu', u'Sha Meng')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Jia Liu', u'Sha Meng')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Jia Liu', u'Peng Yu')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Frank Seide', u'Sha Meng'))]

In [0]:
#Check if the rows in the tables are unique - do we have any repeats? If so then we want to avoid counting pair-title values multiple times
book_authors_join.map(lambda row: ((row[0], row[1]), 1)).reduceByKey(lambda x, y: x+y).filter(lambda row: row[1] != 1).take(10)

[((u'Introduction.', (u'James Ohene-Djan', u'Robert Logcher')), 2),
 ((u'Adaptive Video on Demand.', (u'Amir Herzberg', u'Sudhanshu Aggarwal')),
  4),
 ((u'Minitrack Introduction.', (u'Jrg M. Haake', u'Judith Gebauer')), 2),
 ((u'Minitrack Introduction.', (u'Robert O. Briggs', u'Samuli Pekkola')), 2),
 ((u'Minitrack Introduction.',
   (u'Christopher P. Holland', u'Mark N. Frolick')),
  2),
 ((u'A Semantic Data Grid for Satellite Mission Quality Analysis.',
   (u'Manuel Snchez-Gestido', u'scar Corcho')),
  4),
 ((u'3.', (u'Gershon Elber', u'Yuefei Zhu')), 2),
 ((u'Minitrack Introduction.', (u'Pirkko Walden', u'Scott McCoy')), 4),
 ((u'BLOG: Probabilistic Models with Unknown Objects.',
   (u'Andrey Kolobov', u'Brian Milch')),
  4),
 ((u'Minitrack Introduction.', (u'Kenneth J. Trimmer', u'Marios Koufaris')),
  2)]

In [0]:
#it appears that some pair-title combinations appear more than once therefore we group by these combinations before mapping the pairs to values of 1
count = book_authors_join.map(lambda row: ((row[0], row[1]), 1)).groupByKey().mapValues(list).map(lambda row: (row[0][1], 1))
count.take(5)

[((u'Don DeSota', u'Sameh Sharkawi'), 1),
 ((u'Xiao-Yuan Jing', u'Yong-Chuan Zhang'), 1),
 ((u'Guy-Ren Perrin', u'Wolfgang Karl'), 1),
 ((u'J. W. Park', u'Y. S. Moon'), 1),
 ((u'Shuichi Sakai', u'Toshitsugu Yuba'), 1)]

In [0]:
#finally we reduce by author pairs, summing up the mapped values and then sorting the sums in descending order to give the top 10 author pairs
count.reduceByKey(lambda x, y: x+y).sortBy(lambda row: row[1], ascending = False).take(10)

[((u'Irith Pomeranz', u'Sudhakar M. Reddy'), 246),
 ((u'Amr El Abbadi', u'Divyakant Agrawal'), 161),
 ((u'Makoto Takizawa', u'Tomoya Enokido'), 137),
 ((u'Didier Dubois', u'Henri Prade'), 122),
 ((u'Elizabeth Chang', u'Tharam S. Dillon'), 115),
 ((u'Mary Jane Irwin', u'Narayanan Vijaykrishnan'), 107),
 ((u'Mahmut T. Kandemir', u'Mary Jane Irwin'), 100),
 ((u'Chun Chen', u'Jiajun Bu'), 99),
 ((u'Shojiro Nishio', u'Takahiro Hara'), 96),
 ((u'Filip De Turck', u'Piet Demeester'), 90)]

Part 2
create table in pySpark with schema

In [0]:
filename = 'gs://dora-bucket/author-large.txt'

from pyspark.sql.types import *

schema = StructType([
    StructField("author", StringType(), True),    
    StructField("journal", StringType(), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True)
])

author_large = spark.read.csv(filename, 
                    header='false', schema=schema, sep='\t')
author_large.createOrReplaceTempView("author_large")

In [0]:
spark.sql("""with all_combos as (select CONCAT(a2.author, ', ', a1.author) author_pair, a1.title
        from author_large a1 join author_large a2 on a1.title=a2.title where a1.author>a2.author)
        select author_pair, count(distinct title) count from all_combos group by author_pair order by count desc""").show(10, truncate = False)

+----------------------------------------+-----+
|author_pair                             |count|
+----------------------------------------+-----+
|Irith Pomeranz, Sudhakar M. Reddy       |246  |
|Amr El Abbadi, Divyakant Agrawal        |161  |
|Makoto Takizawa, Tomoya Enokido         |137  |
|Didier Dubois, Henri Prade              |122  |
|Elizabeth Chang, Tharam S. Dillon       |115  |
|Mary Jane Irwin, Narayanan Vijaykrishnan|107  |
|Mahmut T. Kandemir, Mary Jane Irwin     |100  |
|Chun Chen, Jiajun Bu                    |99   |
|Shojiro Nishio, Takahiro Hara           |96   |
|Filip De Turck, Piet Demeester          |90   |
+----------------------------------------+-----+
only showing top 10 rows



Part 3

Need to transform JSON into a dataframe using Hive
Lets look at the first line of the JSON file
Download JAR file

Code for download
SSH:
gsutil cp gs://dora-bucket/hive-json-serde.jar hive-json-serde.jar
gsutil cp gs://dora-bucket/yelp_academic_dataset_user.json yelp_academic_dataset_user.json


 "user_id": "myql3o3x22_ygECb8gVo7A",
  "name": "Vivian",
  "review_count": 80,
  "yelping_since": "2009-06-27",
  "friends" : [...],
  "useful": 117,
  "funny": 28,
  "cool": 104,
  "fans": 34,
  "elite": [
    "None"
  ],
  "average_stars": 4.29,
  "compliment_hot": 64,
  "compliment_more": 4,
  "compliment_profile": 5,
  "compliment_cute": 11,
  "compliment_list": 1,
  "compliment_note": 97,
  "compliment_plain": 129,
  "compliment_cool": 144,
  "compliment_funny": 144,
  "compliment_writer": 17,
  "compliment_photos": 24,
  "type": "user"

CREATE EXTERNAL TABLE IF NOT EXISTS my_table (
 user_id STRING,
  name STRING,
  review_count INT,
  yelping_since STRING,
  friends ARRAY<STRING>,
  useful INT,
  funny INT,
  cool INT,
  fans INT,
  elite ARRAY<STRING>,
  verage_stars FLOAT,
  compliment_hot INT,
  compliment_more INT,
  compliment_profile INT,
  compliment_cute INT,
  compliment_list INT,
  compliment_note INT,
  compliment_plain INT,
  compliment_cool INT,
  compliment_funny INT,
  compliment_writer INT,
  compliment_photos INT,
  type STRING ) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.contrib.serde2.JsonSerde' 
LOCATION 'path/to/my_table/';
